In [9]:
#P-GCS
from numpy import *
import matplotlib.pyplot as plt
from obj_funcs_lib import *

from scipy.optimize import minimize
import scipy
import numpy as np
np.set_printoptions(precision=2)
import datetime



def t(x):
    return np.square( np.tan(np.pi*x-np.pi/2) )
def T(x):
    return (np.pi/2-np.arctan( np.sqrt(x) ) )/np.pi
def s(x,a,b):
    return ((np.cos(2*np.pi*x-np.pi)+1)/2)*(T(b)-T(a))+T(a)
def w(x,a,b):
    return t(s(x,a,b))


def optimize_with_pgcs(
    start_point, 
    objective,
    target=0.05, 
    w_a=0, 
    w_b=np.inf, 
    crunch_period=5000, 
    N=700000
):
    """
    Optimizes a given `objective` function starting from `start_point` with options for
    target value, step size parameters w_a, w_b, crunching period, and the number of iterations N.
    """
    sd = w_a
    current_pos = np.array(start_point)
    current_obj_value = objective(current_pos)
    crunch_step = 0
    w_cache = np.zeros(crunch_period)

    # Precompute w_cache
    for i in range(crunch_period):
        w_cache[i] = w(i / crunch_period, w_a, w_b)

    i = 0
    while i < N:
        if i == 0:
            powell_result = minimize(objective, current_pos, method='Powell')
            current_pos = powell_result.x
            current_obj_value = powell_result.fun
        temp_pos = current_pos + np.random.normal(0, sd, np.array(start_point).shape)
        temp_obj_value = objective(temp_pos)
        if temp_obj_value < current_obj_value:
            temp_powell = minimize(objective, temp_pos, method='Powell')
            temp_powell_pos = temp_powell.x
            temp_powell_value = temp_powell.fun
            if temp_powell_value < temp_obj_value:
                temp_obj_value = temp_powell_value
                temp_pos = temp_powell_pos
            current_pos = temp_pos
            current_obj_value = temp_obj_value
        else:
            crunch_step += 1
        if current_obj_value < target:
            break
        if crunch_step % crunch_period == 0:
            crunch_step = 0
        sd = w_cache[crunch_step]
        i += 1

    print(current_pos)
    print(objective(current_pos))
    return current_pos, objective(current_pos)



import time

results = {}
for name, obj_func in OBJ_FUNCS.items():
    print(f"Optimizing {name}...")
    start_time = time.time()
    result = optimize_with_pgcs(
        start_point=[200]*3,
        objective=obj_func,
    )
    elapsed = time.time() - start_time
    print(f"Time taken for {name}: {elapsed:.2f} seconds")
    results[name] = result





Optimizing sphere...
[-1.99e-13 -2.56e-13 -2.56e-13]
1.704444426256727e-25
Time taken for sphere: 0.04 seconds
Optimizing rosenbrock...
[0.92 0.85 0.73]
[0.03]
Time taken for rosenbrock: 15.08 seconds
Optimizing rastrigin...
[-8.67e-11 -3.67e-09 -3.67e-09]
0.0
Time taken for rastrigin: 0.04 seconds
Optimizing ackley...
[ 4.90e-12 -1.32e-12 -1.83e-12]
[1.25e-11]
Time taken for ackley: 8.06 seconds
Optimizing griewank...
[-9.42e+00  5.28e-07  5.43e+00]
[0.03]
Time taken for griewank: 1.06 seconds
